In [ ]:
%cd /content/
%rm -rf /content/acl20-ref-free-eval
%cd /content/

/content
/content


In [ ]:
#Clone SUPERT Repo
!git clone https://github.com/yg211/acl20-ref-free-eval.git

Cloning into 'acl20-ref-free-eval'...
remote: Enumerating objects: 380, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 380 (delta 25), reused 69 (delta 18), pack-reused 294
Receiving objects: 100% (380/380), 197.49 KiB | 2.38 MiB/s, done.
Resolving deltas: 100% (141/141), done.


In [ ]:
#Examine ECTSum Directory
!ls acl20-ref-free-eval -l

total 60
drwxr-xr-x 4 root root 4096 Aug  3 17:59 data
-rw-r--r-- 1 root root 1428 Aug  3 17:59 evaluate_summary.py
-rw-r--r-- 1 root root 6747 Aug  3 17:59 generate_summary_ga.py
-rw-r--r-- 1 root root 2511 Aug  3 17:59 generate_summary_rl.py
-rw-r--r-- 1 root root 4347 Aug  3 17:59 README.md
drwxr-xr-x 2 root root 4096 Aug  3 17:59 ref_free_metrics
-rw-r--r-- 1 root root  457 Aug  3 17:59 requirements.txt
-rwxr-xr-x 1 root root  231 Aug  3 17:59 resources.py
drwxr-xr-x 2 root root 4096 Aug  3 17:59 rouge
drwxr-xr-x 6 root root 4096 Aug  3 17:59 sentence_transformers
drwxr-xr-x 3 root root 4096 Aug  3 17:59 summariser
drwxr-xr-x 2 root root 4096 Aug  3 17:59 summ_eval
drwxr-xr-x 2 root root 4096 Aug  3 17:59 utils


In [ ]:
#Connect to Google Drive
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Examine MyDrive Directory
!ls drive/MyDrive/w266-project/ -l

total 455318
-rw------- 1 root root    204342 Jul 30 21:37  ECTBPS_Extract.ipynb
-rw------- 1 root root  15046328 Jul 21 00:33  ECTBPS_Paraphrase.ipynb
-rw------- 1 root root    221497 Jul 21 00:33  ECTBPS_Paraphrase_T5.ipynb
-rw------- 1 root root    215680 Jul 21 00:33  ECTBPS_Pegasus_Paraphrase.ipynb
-rw------- 1 root root     30784 Jul 17 02:49  ECTQA_Compute_Metrics.ipynb
drwx------ 2 root root      4096 Jul 28 17:53  ECTQA-Models
-rw------- 1 root root       175 Aug  1 23:49 'Human Evaluation of Summaries.gsheet'
drwx------ 2 root root      4096 Jul 29 20:57  LEDmodel
drwx------ 2 root root      4096 Jun  7 01:04  lit-review
drwx------ 2 root root      4096 Jun 23 00:24  new_data
-rw------- 1 root root       175 Aug  3 18:05  outline-final-paper.gdoc
-rw------- 1 root root       175 Jul 17 01:48  outline-milestone-paper.gdoc
drwx------ 2 root root      4096 Jul  7 18:09  outputs_ext
drwx------ 2 root root      4096 Jul  9 23:23  outputs_para
drwx------ 2 root root      4096 Jul  

# Set Up SUPERT Single Doc Summaries

In [ ]:
%cd acl20-ref-free-eval

/content/acl20-ref-free-eval


In [ ]:
!pip install pytorch_transformers

In [ ]:
# Import Libraries and Configurations
from ref_free_metrics.supert import Supert
from utils.data_reader import CorpusReader
import nltk

nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#DO NOT RUN BELOW
#%cp /content/drive/MyDrive/w266-project/new_data/final/final/*/*.txt /content/drive/MyDrive/w266-project/supert/input_docs -f

#Use Text File Processing

In [ ]:
def getPredData(dataPath):
  predPath = f'dataPath'
  testFiles = [file for file in os.listdir(dataPath)]
  PredData = []
  for file in sorted(testFiles):
    dataset = []
    doc_in = open(f'{dataPath}/{file}', 'r', encoding='utf8')
    doc_lines = [line.strip() for line in doc_in.readlines()]
    for line in doc_lines:
      text_to_paraphrase = line.strip().lower()
      dataset.append(text_to_paraphrase)
    PredData.append('\n'.join(dataset) + '\n')

  return PredData

In [ ]:
predpath = '/content/drive/MyDrive/w266-project/supert/input_docs'
source_docs = getPredData(predpath)

In [ ]:
#import nltk
#nltk.download('punkt')
#from nltk.tokenize import sent_tokenize
#import os
#testFiles = [file for file in os.listdir(predpath)]
#for file in sorted(testFiles):
#  print(file)
#all_sents = []

#for dd in range(len(source_docs)):
#    dname = sent_tokenize(source_docs[dd])
#    all_sents.append(dname)

In [ ]:
#[len(i) for i in all_sents]

In [ ]:
print(len(source_docs))
print(len(source_docs[0].split(' ')))
print(source_docs[0][0])

182
6871
r


# PARSE_DOCS -> GET_WEIGHTS ATTEMPT

In [ ]:
from sentence_transformers import SentenceTransformer, models
# loading bar = download screen for bert-large-nli-stsb-mean-tokens
bert_model = SentenceTransformer('bert-large-nli-stsb-mean-tokens') #uses bert-large-nli sentence transformer
#mpnet_model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
from nltk.tokenize import sent_tokenize

## FOR LOOP TO PRODUCE LIST OF SENTENCES BY DOC AND STORE IN DICTIONARY
all_sents = []
sent_index = {}
cnt = 0

for dd in range(len(source_docs)):
    dname = sent_tokenize(source_docs[dd])#.split('.')
    doc_len = len(source_docs[dd])
    sent_index[dd] = {'doc': dname, 'text': dname, 'inside_doc_idx': dd, 'doc_len': doc_len,
                                  'inside_doc_position_ration': dd * 1. / doc_len}
    #for i, sent in enumeratesource_docs:
    #  sent_index[cnt] = {'doc': dname, 'text': sent, 'inside_doc_idx': i, 'doc_len': doc_len,
    #                              'inside_doc_position_ration': i * 1. / doc_len}
    all_sents.append(dname)

#all_vecs = bert_model.encode(all_sents)

In [ ]:
len(all_sents[0])

387

In [ ]:
len(sent_index)

182

In [ ]:
#USE BERT MODEL TO ENCODE LIST OF SENTENCES IN DOCUMENT
#NEED TO TRAVERSE WITH FOR LOOP
all_vecs = []

for i in range(len(all_sents)):
  x = bert_model.encode(all_sents[i])
  all_vecs.append(x)

In [ ]:
len(all_vecs)

182

In [ ]:
len(sent_index)

182

In [ ]:
#USE GET_WEIGHTS script to obtain weights by sentence in each doc to produce pseudo_reference
from ref_free_metrics.similarity_scorer import get_weights

sents_weights = []

for i in range(len(all_vecs)):
  x = get_weights(sent_index, all_vecs[i], metric = 'global_cluster') #use global_cluster on common phrases throughout ECTQA dataset
  sents_weights.append(x)


global cluster, pseudo-ref sent num 44
global cluster, pseudo-ref sent num 39
global cluster, pseudo-ref sent num 45


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


global cluster, pseudo-ref sent num 67
global cluster, pseudo-ref sent num 30
global cluster, pseudo-ref sent num 30
global cluster, pseudo-ref sent num 47
global cluster, pseudo-ref sent num 61


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


global cluster, pseudo-ref sent num 50


...

global cluster, pseudo-ref sent num 56
global cluster, pseudo-ref sent num 31
global cluster, pseudo-ref sent num 51


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


global cluster, pseudo-ref sent num 51
global cluster, pseudo-ref sent num 31


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


global cluster, pseudo-ref sent num 54
global cluster, pseudo-ref sent num 24
global cluster, pseudo-ref sent num 44
global cluster, pseudo-ref sent num 54
global cluster, pseudo-ref sent num 33
global cluster, pseudo-ref sent num 27
global cluster, pseudo-ref sent num 35
global cluster, pseudo-ref sent num 32
global cluster, pseudo-ref sent num 37
global cluster, pseudo-ref sent num 69


In [ ]:
type(sent_index[0]['text'])

list

In [ ]:
type(sents_weights[0])

list

In [ ]:
print(len(sents_weights[0]))
print(sents_weights[0][0])
print(len(sent_index[0]['text']))
print(sent_index[0]['text'][0])

387
0.0
387
ross seymore, deutsche bank.


In [ ]:
sent_index[0]['text']

['ross seymore, deutsche bank.',
 'hello.',
 'thanks for letting me ask a question.',
 "first and foremost, b.k., if you can go into a little bit more about what led to the restructuring announcement, not only the one today, but there's been significant changes in the senior management over the last three months.",
 "so if you could just talk us through some of what's changed strategically in the company that's led to such a large amount of turnover so recently?",
 'sure.',
 "i mean, if you take a look at it, so let's step back for a second and just talk about why are we doing the restructuring now, and then we can talk about some of the specifics beyond that.",
 "so we've talked about this transformation, that we're moving from a client-centric, you know intel's been typically known as the pc company, to a company that is focused more and more on a much broader set of products, and really focused around the cloud, right?",
 'and the cloud, and all the connected devices that connect to

In [ ]:
#pseudo_refs = [sent_index[idx]['text'][i] for idx in range(len(sents_weights)) for i, weight in enumerate(sents_weights[idx]) if weight == 1]

pseudo_refs = []
for idx in range(len(sents_weights)):
    pseudo_refs.append([sent_index[idx]['text'][i] for i in range(len(sents_weights[idx])) if sents_weights[idx][i] == 1])

In [ ]:
print(sents_weights[0])
print(sum(x > 0 for x in sents_weights[0]))
print(len(pseudo_refs[0]))

[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,

In [ ]:
pseudo_refs[0]

['hello.',
 'sure.',
 "and so what this effort around the restructuring is, is to say, it's time now to try, and we've made enough progress, right?",
 'and the one that stacy and i have been working for several months, actually several quarters, on what does he do next, how do we growth his both exposure to other parts of the company and also the rest of the company see his leadership style.',
 'this is going to be an orderly transition.',
 'my follow-up would be the full-year guidance of mid single-digit growth.',
 "what we're projecting for the year is overall growth, and we think we'll see growth in the data center, growth in iot, growth in memory that's offsetting some, oh and of course we've added in altera now into our product family, and so you take that and that's offsetting some of the weakness that we're seeing in the pc market.",
 'great, thank you.',
 'you know, we think that we can become more focused.',
 'you did a great job explaining exactly why we believe that the data

#Write Pseudo Ref Summaries to .txt files

In [ ]:
#Write files to a folder
import os
folder_path = "/content/drive/MyDrive/w266-project/supert/input_docs"
file_names = [file for file in os.listdir(folder_path)]
file_names.sort()

In [ ]:
file_names[0]

'2016-Apr-19-INTC.txt'

In [ ]:
def write_pseudo_refs_to_files(pseudo_refs, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, ref_summary in enumerate(pseudo_refs):
        file_path = os.path.join(output_folder, f"{file_names[i][:-4]}_ref.txt")
        with open(file_path, 'w', encoding='utf-8') as file:
            for sentence in ref_summary:
                file.write(sentence + '\n')

output_folder = "/content/drive/MyDrive/w266-project/supert/ref_summaries"
write_pseudo_refs_to_files(pseudo_refs, output_folder)

#Evaluate SUPERT-generated summaries

In [ ]:
sourcePath = '/content/drive/MyDrive/w266-project/supert/input_docs'
source_docs = getPredData(sourcePath)

pseudoPath = '/content/drive/MyDrive/w266-project/supert/ref_summaries'
pseudo_refs = getPredData(pseudoPath)

In [ ]:
#print(source_docs[0])
#print(pseudo_refs[0])

In [ ]:
!pip install evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=22ff2d2310d10d3c0ced7d965afc6941e700b53b246e32ef81d402a7440ae1b3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
def getScores(dataset1, dataset2):
  scores = []
  for i in range(len(dataset1)):
    scores1 = rouge.compute(
        predictions=[dataset1[i]], references=[dataset2[i]]
        )
    scores.append(scores1)
  rouge1 = []
  rouge2 = []
  rougeL = []
  rougeLsum = []
  for i in range(len(scores)):
    rouge1.append(scores[i]['rouge1'])
    rouge2.append(scores[i]['rouge2'])
    rougeL.append(scores[i]['rougeL'])
    rougeLsum.append(scores[i]['rougeLsum'])

  rouge1Avg = sum(rouge1) / len(rouge1)
  rouge2Avg = sum(rouge2) / len(rouge2)
  rougeLAvg = sum(rougeL) / len(rougeL)
  rougeLsumAvg = sum(rougeLsum) / len(rougeLsum)
  return 'rouge 1 Avg', rouge1Avg, 'rouge 2 Avg', rouge2Avg, 'rouge L Avg', rougeLAvg, 'rouge LSum Avg', rougeLsumAvg

In [ ]:
rougeSupert = getScores(pseudo_refs, source_docs)

In [ ]:
rougeSupert

('rouge 1 Avg',
 0.2219277730148946,
 'rouge 2 Avg',
 0.2140156790808566,
 'rouge L Avg',
 0.2219277730148946,
 'rouge LSum Avg',
 0.22190417122922582)

In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.4 MB/s eta 0:00:00


In [ ]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=source_docs, references=pseudo_refs, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print('The average precision is', sum(results['precision'])/len(results['precision']), '\nThe average recall is', sum(results['recall'])/len(results['recall']), '\nThe average F1 score is', sum(results['f1'])/len(results['f1']))

The average precision is 0.84316545855868 
The average recall is 0.8443833560079008 
The average F1 score is 0.8437503631953355


# Compute newsroom statistics

In [ ]:
sourcePath = '/content/drive/MyDrive/w266-project/supert/input_docs'
source_docs = getPredData(sourcePath)

pseudoPath = '/content/drive/MyDrive/w266-project/supert/ref_summaries'
pseudo_refs = getPredData(pseudoPath)

In [ ]:
!pip install nltk
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=e25d8ff54ec6d5a1026e3c0556b1ed51da6faa501fe5de52f2390bd0ac8404ce
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
1-len(pseudo_refs[0]) / len(source_docs[0])

0.8743779491014104

In [ ]:
from nltk.tokenize import sent_tokenize
from rouge_score import rouge_scorer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def calculate_scores(input_docs, summaries):
    assert len(input_docs) == len(summaries), "Input documents and summaries lists must have the same length."

    # Initialize the Rouge scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    coverage_scores = []
    density_scores = []
    compression_scores = []

    for doc, summ in zip(input_docs, summaries):
        # Calculate Coverage score
        coverage_score = scorer.score(summ, doc)['rougeL'].fmeasure
        coverage_scores.append(coverage_score)

        # Calculate Density score
        num_sentences = len(sent_tokenize(doc))
        num_summary_sentences = len(sent_tokenize(summ))**2
        density_score = num_summary_sentences / num_sentences
        density_scores.append(density_score)

        # Calculate Compression score
        compression_score = 1-(len(summ) / len(doc))
        compression_scores.append(compression_score)

    return coverage_scores, density_scores, compression_scores

In [ ]:
coverage_scores, density_scores, compression_scores = calculate_scores(source_docs, pseudo_refs)

print("Coverage scores:", coverage_scores)
print("Density scores:", density_scores)
print("Compression scores:", compression_scores)


Coverage scores: [0.22211503677800554, 0.20282828282828286, 0.23320220298977185, 0.18418079096045198, 0.2239067055393586, 0.25253585169639736, 0.21472296342803326, 0.1796516956920257, 0.2086444007858546, 0.2537524780515435, 0.23231338920288447, 0.23208191126279862, 0.18635437881873726, 0.23986044483209767, 0.19755725190839693, 0.23728372577078183, 0.1957765068191817, 0.22323324234152261, 0.20488179896213723, 0.2257197696737044, 0.21256589257729022, 0.2329964490576345, 0.22783588093322604, 0.2060549166862239, 0.20908199934661875, 0.19504459861248763, 0.2137135533482858, 0.18636363636363637, 0.2206081496983873, 0.1957849725106903, 0.21241450509743195, 0.2401315789473684, 0.21171097700212038, 0.19925489724792692, 0.2229224103977944, 0.2517609634174051, 0.2091869743368557, 0.2055005500550055, 0.20346562325321407, 0.21970787903723513, 0.25400898011545864, 0.21858894615247912, 0.21931838248278301, 0.24080749819754863, 0.21416288885415039, 0.21446433930179054, 0.20258566587172083, 0.237823211

In [ ]:
average_coverage_score = sum(coverage_scores) / len(coverage_scores)
average_density_score = sum(density_scores) / len(density_scores)
average_compression_score = sum(compression_scores) / len(compression_scores) * 100

print("Average Coverage score:", average_coverage_score)
print("Average Density score:", average_density_score)
print("Average Compression score:", average_compression_score)

Average Coverage score: 0.2219277730148946
Average Density score: 4.815194302532119
Average Compression score: 87.49750539906694
